Click here to open Google Colab notebook version [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1AQVulyKLqi2d6v2HtcYgpE9pP7v8PXgu?usp=sharing)

This notebook introduces a quick tutorial on how to use **AutoGen** 🤖 to implement the following features:
*   🧑‍💻🧑‍🔧 Automatically generate code and debug.
*   🧰 Use user's functions as tools.
*   🔍 Keep LLM's knowledge up-to-date using search tools.
*   🚀 Implement an LLM-based multi-agent system with Group Chat.

# ⚙️ Setup

In [ ]:
openai_api_key = "<YOUR OPENAI_API_KEY>"

# 🧑‍💻🧑‍🔧 Automatically generate code and debug.

In [ ]:
import autogen
from autogen.coding import LocalCommandLineCodeExecutor


config_list = [{"model": "gpt-4", "api_key": openai_api_key}]
llm_config = {
    "cache_seed": 41,
    "config_list": config_list,
    "temperature": 0,
}

# assistant will generate code to solve problems
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

# user_proxy will give problem, execute code from assistant, and return output
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        # the executor to run the generated code
        "executor": LocalCommandLineCodeExecutor(work_dir="coding"),
    },
)

In [ ]:
# the assistant receives a message from the user_proxy, which contains the task description
chat_res = user_proxy.initiate_chat(
    assistant,
    message="""
    Give me python function phone_regex that automatically finds phone numbers.
    The function gets a text as an input and return the phone numbers inside it,
    if not, return None.
    Inside the main python file, please add these unit tests:
    assert phone_regex("(12)1231231234") == "(12)1231231234"
    assert phone_regex("12123)1231234") is None
    assert phone_number("12(1231231234") == "1231231234"
    """,
    summary_method="reflection_with_llm",
)

user_proxy (to assistant):


    Give me python function phone_regex that automatically finds phone numbers.
    The function gets a text as an input and return the phone numbers inside it,
    if not, return None. 
    Inside the main python file, please add these unit tests:
    assert phone_regex("(12)1231231234") == "(12)1231231234"
    assert phone_regex("12123)1231234") is None
    assert phone_number("12(1231231234") == "1231231234"
    

--------------------------------------------------------------------------------
assistant (to user_proxy):

Sure, I can help with that. Here is a Python script that defines a function `phone_regex` to find phone numbers in a given text. It uses regular expressions to match the phone number pattern. The script also includes the unit tests you requested.

```python
# filename: phone_regex.py

import re

def phone_regex(text):
    # The regular expression pattern for a phone number
    pattern = r'\(?\d{2}\)?\d{10}'
    match = re.search(pattern,

# 🧰 Use user's functions as tools.

In [ ]:
import re


def phone_regex(text):
    # The regular expression pattern for a phone number
    pattern = r'\(?\d{2}\)?\d{10}'
    match = re.search(pattern, text)
    return match.group() if match else None

In [ ]:
phone_regex("My phone number is (12)1231231234")

'(12)1231231234'

In [ ]:
config_list = [{"model": "gpt-4", "api_key": openai_api_key}]
llm_config = {
    "cache_seed": 41,
    "config_list": config_list,
    "temperature": 0,
}

system_message = """"
    For phone extracting tasks, only use the functions you have been provided with.
    Reply TERMINATE when the task is done.
"""
assistant = autogen.AssistantAgent(
    name="assistant",
    system_message=system_message,
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "executor": LocalCommandLineCodeExecutor(work_dir="coding"),
    },
)


# register function so that user_proxy can access and run
@user_proxy.register_for_execution()
@assistant.register_for_llm(description="Extract a phone number from the given text")
def phone_number_extracting(
        text: Annotated[str, "a text to extract a phone number"]
    ) -> str:
    return phone_regex(text)

In [ ]:
# the assistant receives a message from the user_proxy, which contains the task description
chat_res = user_proxy.initiate_chat(
    assistant,
    message="""
    Extract phone number from the following text:
        "My phone number is (12)1231231234"
    """,
    summary_method="reflection_with_llm",
)

user_proxy (to assistant):


    Extract phone number from the following text:
        "My phone number is (12)1231231234"
    

--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested tool call (call_SEqxWfCxw4lQoVxfPdQsPmRM): phone_number_extracting *****
Arguments: 

{
"text": "My phone number is (12)1231231234"
}
****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION phone_number_extracting...
user_proxy (to assistant):

user_proxy (to assistant):

***** Response from calling tool (call_SEqxWfCxw4lQoVxfPdQsPmRM) *****
(12)1231231234
**********************************************************************

--------------------------------------------------------------------------------
assistant (to user_proxy):

The extracted phone number is (12)1231231234.

---------------

# 🔍 Keep LLM's knowledge up-to-date using search tools.

In [ ]:
bing_api_key = "<YOUR_BING_API_KEY>"

In [ ]:
summarizer_config_list = [{"model": "gpt-3.5-turbo", "api_key": openai_api_key}]

summarizer_llm_config = {
    "timeout": 600,
    "cache_seed": 41,
    "config_list": summarizer_config_list,
    "temperature": 0,
}

In [ ]:
from autogen.agentchat.contrib.web_surfer import WebSurferAgent


web_surfer = WebSurferAgent(
    name="web_surfer",
    llm_config={
        "cache_seed": 41,
        "config_list": config_list,
        "temperature": 0,
    },
    summarizer_llm_config=summarizer_llm_config,
    browser_config={"viewport_size": 4096, "bing_api_key": bing_api_key},
)


user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: True,
)

In [ ]:
task1 = """
Search the web for tutorial with code about Object Detection using newest frameworks
"""

chat_res = user_proxy.initiate_chat(web_surfer, message=task1)

user_proxy (to web_surfer):


Search the web for tutorial with code about Object Detection using newest frameworks


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION informational_web_search...
web_surfer (to user_proxy):

Address: bing: Object Detection tutorial with code using newest frameworks
Title: Object Detection tutorial with code using newest frameworks - Search
Viewport position: Showing page 1 of 1.
A Bing search for 'Object Detection tutorial with code using newest frameworks' found 10 results:

## Web Results
1. [The practical guide for Object Detection with YOLOv5 algorithm](https://towardsdatascience.com/the-practical-guide-for-object-detection-with-yolov5-algorithm-74c04aac4843)
One of the most popular algorithms to date for real-time object detection is YOLO (You Only Look Once), initially proposed by Redmond et. al [1]. In this tutorial you will learn to perform an end-to-end ob

In [ ]:
task2 = """
    Summarize these results based on used python libraries or used techonologies.
    Remember to include a link for each summarized content.
"""

chat_res =  user_proxy.initiate_chat(web_surfer, message=task2, clear_history=False)

user_proxy (to web_surfer):


    Summarize these results based on used python libraries or used techonologies.
    Remember to include a link for each summarized content.


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
web_surfer (to user_proxy):

Based on the search results, the following technologies and Python libraries are commonly used in the latest object detection tutorials:

1. **YOLO (You Only Look Once)**: This is a popular real-time object detection algorithm. The latest versions, YOLOv5, YOLOv8, and YOLOv9, are mentioned in several tutorials. The YOLOv5 implementation is developed by Ultralytics. [Link](https://towardsdatascience.com/the-practical-guide-for-object-detection-with-yolov5-algorithm-74c04aac4843)

2. **TensorFlow**: TensorFlow is a free and open-source software library for machine learning and artificial intelligence. It is used in a tutorial provided by TensorFlow Hub for object detection. [Link

# 🚀 Implement an LLM-based multi-agent system with Group Chat.

In [ ]:
bing_api_key = "<YOUR_BING_API_KEY>"

In [ ]:
summarizer_config_list = [{"model": "gpt-3.5-turbo", "api_key": openai_api_key}]

summarizer_llm_config = {
    "timeout": 600,
    "cache_seed": 41,
    "config_list": summarizer_config_list,
    "temperature": 0,
}

In [ ]:
from autogen.agentchat.contrib.web_surfer import WebSurferAgent


config_list = [{"model": "gpt-3.5-turbo", "api_key": openai_api_key}]
llm_config = {
    "cache_seed": 41,
    "config_list": config_list,
    "temperature": 0,
}

web_surfer = WebSurferAgent(
    name="web_surfer",
    system_message="""
        Search projects tutorials which fits with the given project
    """,
    llm_config={
        "cache_seed": 41,
        "config_list": [{"model": "gpt-4", "api_key": openai_api_key}],
        "temperature": 0,
    },
    summarizer_llm_config=summarizer_llm_config,
    browser_config={"viewport_size": 4096, "bing_api_key": bing_api_key},
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: True,
)

project_planner = autogen.AssistantAgent(
    name="project_planner",
    system_message="""
        Read requirements and give a name of REAL-WORLD PROBLEM SOLVING PROJECT to meet these requirements
    """,
    llm_config=llm_config,
)

In [ ]:
groupchat = autogen.GroupChat(agents=[user_proxy, project_planner, web_surfer], messages=[], max_round=5)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
requirements = """
What we're looking for:

- Currently studying Computer Science, AI/ML, or something similar.
- Good grasp of Deep Learning, Computer Vision with hands-on experience using machine learning frameworks like TensorFlow or PyTorch.
- Good English skills, able to understand and discuss technical topics.
- Strong at analyzing and solving problems.
- Knows basic git commands.
"""

In [ ]:
chat_res = user_proxy.initiate_chat(
    manager, message=f"Research and give me a project so I can fit with this job requirements: {requirements}"
)

user_proxy (to chat_manager):

Research and give me a project so I can fit with this job requirements: 
What we're looking for:

- Currently studying Computer Science, AI/ML, or something similar.
- Good grasp of Deep Learning, Computer Vision with hands-on experience using machine learning frameworks like TensorFlow or PyTorch.
- Good English skills, able to understand and discuss technical topics.
- Strong at analyzing and solving problems.
- Knows basic git commands.


--------------------------------------------------------------------------------
project_planner (to chat_manager):

Real-World Problem Solving Project Name: "Automated Plant Disease Detection System"

Description:
The project involves developing an automated system using deep learning and computer vision techniques to detect and classify plant diseases from images. This project will leverage machine learning frameworks like TensorFlow or PyTorch to train a model that can accurately identify various plant diseases bas